## Testing by reconstructing image

In [9]:
import os
import re
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

def reconstruct_from_patches(patch_dir, skipped_csv_path, output_path, patch_size=512):
    # Load skipped coordinates
    skipped_df = pd.read_csv(skipped_csv_path, usecols=['x', 'y'])

    # Parse all available patch filenames
    patch_files = [f for f in os.listdir(patch_dir) if f.endswith('.png')]
    coords_dict = {}

    # Regular expression to extract x, y from filename
    pattern = re.compile(r'_x(\d+)_y(\d+)_patch\d+\.png')

    for file in patch_files:
        match = pattern.search(file)
        if match:
            x, y = map(int, match.groups())
            coords_dict[(x, y)] = file

    # Combine available + skipped coordinates
    all_coords = set(coords_dict.keys()).union(set([tuple(row) for row in skipped_df[['x', 'y']].values]))

    # Determine full canvas size
    all_x = [x for x, _ in all_coords]
    all_y = [y for _, y in all_coords]
    min_x, max_x = min(all_x), max(all_x)
    min_y, max_y = min(all_y), max(all_y)

    width = max_x - min_x + patch_size
    height = max_y - min_y + patch_size



    n_cols = (width // patch_size)
    n_rows = (height // patch_size)

    print(f"🧱 Reconstructing image of size: {width} x {height} ({n_cols} cols x {n_rows} rows)")

    # Create blank white canvas
    canvas = Image.new('RGB', (width, height), color='white')

    for (x, y) in tqdm(all_coords, desc='🧩 Placing patches'):
        if (x, y) in coords_dict:
            patch_path = os.path.join(patch_dir, coords_dict[(x, y)])
            patch = Image.open(patch_path).convert('RGB')
        else:
            patch = Image.new('RGB', (patch_size, patch_size), color='white')

        scaled_x = x - min_x
        scaled_y = y - min_y

        canvas.paste(patch, (scaled_x, scaled_y))


    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Save the reconstructed image
    canvas.save(output_path)
    print(f"✅ Reconstructed image saved to: {output_path}")

In [16]:
import os
import re
import numpy as np
from PIL import Image
from tqdm import tqdm

def reconstruct_from_patches(patch_dir, output_path, patch_size=512):
    # Parse all available patch filenames
    patch_files = [f for f in os.listdir(patch_dir) if f.endswith('.png')]
    coords_dict = {}

    # Regular expression to extract x, y from filename
    pattern = re.compile(r'_x(\d+)_y(\d+)_patch\d+\.png')

    for file in patch_files:
        match = pattern.search(file)
        if match:
            x, y = map(int, match.groups())
            coords_dict[(x, y)] = file

    if not coords_dict:
        print("❌ No valid patch filenames found.")
        return

    # Get bounding box
    all_x = [x for x, _ in coords_dict.keys()]
    all_y = [y for _, y in coords_dict.keys()]
    min_x, max_x = min(all_x), max(all_x)
    min_y, max_y = min(all_y), max(all_y)

    width = (max_x - min_x + patch_size)
    height = (max_y - min_y + patch_size)

    n_cols = (width // patch_size)
    n_rows = (height // patch_size)

    print(f"🧱 Reconstructing image of size: {width} x {height} ({n_cols} cols x {n_rows} rows)")

    # Create blank canvas (white background)
    canvas = Image.new('RGB', (width, height), color='white')

    # Traverse full grid
    print("📌 Drawing patches (black for missing)...")
    for row in tqdm(range(n_rows), desc="🧩 Placing patches"):
        for col in range(n_cols):
            x = min_x + col * patch_size
            y = min_y + row * patch_size

            if (x, y) in coords_dict:
                patch_path = os.path.join(patch_dir, coords_dict[(x, y)])
                patch = Image.open(patch_path).convert('RGB')
            else:
                patch = Image.new('RGB', (patch_size, patch_size), color='black')  # skipped = black

            canvas.paste(patch, (x - min_x, y - min_y))

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    canvas.save(output_path)
    print(f"✅ Reconstructed image saved to: {output_path}")

In [29]:
patch_directory=r"D:\Aamir Gulzar\KSA_project2\Cancer-detection-classifier\data_preprocessing\Patches\TCGA-AG-3581_None"
skipped_csv= './FLAT_DIRECTORY/patches/TCGA-AG-3581-01Z-00-DX1.2d8ba8ba-0533-41e5-b0e0-0d49ef408302_skipped_patches.csv'
out_path= './Reconstructed_img/TCGA-AG-3581_MSIH_8.png'

In [30]:
reconstruct_from_patches(
    patch_dir=patch_directory,
    output_path=out_path
)

🧱 Reconstructing image of size: 50688 x 37888 (99 cols x 74 rows)
📌 Drawing patches (black for missing)...


🧩 Placing patches:  11%|██████▊                                                        | 8/74 [00:03<00:28,  2.32it/s]

KeyboardInterrupt



In [31]:
patch_directory=r"D:\Aamir Gulzar\data_preprocessing\downsample_data\TCGA-AG-3581"
skipped_csv= './FLAT_DIRECTORY/patches/TCGA-AG-3581-01Z-00-DX1.2d8ba8ba-0533-41e5-b0e0-0d49ef408302_skipped_patches.csv'
out_path= './Reconstructed_img/TCGA-AG-3581_MSIH_9.png'

In [32]:
reconstruct_from_patches(
    patch_dir=patch_directory,
    output_path=out_path
)

🧱 Reconstructing image of size: 49664 x 51200 (97 cols x 100 rows)
📌 Drawing patches (black for missing)...


🧩 Placing patches: 100%|████████████████████████████████████████████████████████████| 100/100 [00:54<00:00,  1.84it/s]


✅ Reconstructed image saved to: ./Reconstructed_img/TCGA-AG-3581_MSIH_9.png
